In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_parquet('Data/train.parquet')
train_data.S_2 = pd.to_datetime(train_data.S_2)
train_data.shape

(5531451, 190)

In [3]:
IR = pd.read_csv('Data/IR1Y.csv').rename({'DATE': 'S_2'}, axis=1).set_index('S_2', drop=True)
IR.index = pd.to_datetime(IR.index)
IR.DGS1.replace('.', np.NaN, inplace=True)
IR.DGS1 = IR.DGS1.astype(float)

In [4]:
Misery = pd.read_csv('Data/Misery.csv').rename({'DATE': 'S_2'}, axis=1).set_index('S_2', drop=True)
Misery.index = pd.to_datetime(Misery.index)

In [5]:
Sahm = pd.read_csv('Data/SAHMREALTIME.csv').rename({'DATE': 'S_2'}, axis=1).set_index('S_2', drop=True)
Sahm.index = pd.to_datetime(Sahm.index)

In [6]:
s = pd.DataFrame(
    {"S_2": pd.date_range(IR.index.min(), IR.index.max(), freq="D")}
).set_index('S_2', drop=True)

In [7]:
FE_df = pd.concat([s, IR, Misery,Sahm], axis=1)
FE_df.fillna(method='ffill', inplace=True)
FE_df.fillna(method='bfill', inplace=True)

FE_df.rename(columns={'DGS1':'FE1', 'UNRATE_CPIAUCSL_PC1': 'FE2', 'SAHMREALTIME':'FE3'}, inplace=True)
FE_columns = FE_df.columns
FE_df

,FE1,FE2,FE3
S_2,,,
1948-01-01,3.47,13.64209,0.77
1948-02-01,3.47,13.28196,0.77
1948-03-01,3.47,10.81818,0.77
1948-04-01,3.47,12.17273,0.77
1948-05-01,3.47,12.88497,0.77
...,...,...,...
2022-06-12,2.58,12.11641,-0.07
2022-06-13,2.89,12.11641,-0.07
2022-06-14,3.15,12.11641,-0.07


In [8]:
train_merged = pd.merge(train_data, FE_df, how='left', on='S_2')
train_merged.set_index('customer_ID', inplace=True)

In [10]:
new_FE_df = train_merged[FE_columns].groupby('customer_ID').agg(['mean', 'min', 'max'])

In [11]:
new_FE_df.columns = ['_'.join(col) for col in new_FE_df.columns.values]

In [15]:
new_FE_df.columns

Index(['FE1_mean', 'FE1_min', 'FE1_max', 'FE2_mean', 'FE2_min', 'FE2_max',
       'FE3_mean', 'FE3_min', 'FE3_max'],
      dtype='object')

In [13]:
train = pd.read_parquet('Data/train_agg_pca50.parquet')
train.shape

(458913, 634)

In [14]:
pd.concat([train, new_FE_df], axis=1).to_parquet('Data/train_agg_pca50_FE.parquet')